In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import pickle, os, glob
from EEG.info import info_exp
from EEG.converter import converter_mne
from EEG.preprocessing import aligner_exp
import mne
import numpy as np
import sys

In [2]:
root = "C:\\eeg\\01exp\\"
name = "20161129_DBS_001"

def load(path,name):
    with open(path + '/' + name, 'rb') as input:
        data = pickle.load(input)
        return data

In [3]:
test = aligner_exp(root + name)

In [4]:
raw = test.getdata(mode='raw')

In [5]:
raw_train = raw['data']['data_train']
raw_events = raw['data']['states_train']

In [6]:
import neo
import mne

mne_info = mne.create_info(ch_names = raw['info'].chanels_names, 
                               sfreq = raw['info'].frequency,
                               ch_types = raw['info'].ch_types, 
                               montage = raw['info'].montage)

raw_mne = mne.io.RawArray(raw_train, mne_info)
#raw_mne.add_events(raw_events)

In [12]:
def create_events(states_full):
    xa = np.hstack([[0], states_full[0], [0]])
    xa1 = (xa != 0) + 0
    xadf = np.diff(xa1)
    starts = np.where(xadf == 1)[0]
    stops_p1 = np.where(xadf == -1)[0]
    lens = stops_p1 - starts
    codes = states_full[0, starts]
    trl = np.zeros((len(starts), 3))
    start = 0
    min_len = min(lens)
    for i in range(0, len(lens)):
        trl[i] = [start, 0, codes[i]]
        start = start + lens[i]
    return trl.astype(int)

events = create_events(raw_events)

In [15]:
def get_labels(labels_names):
    labels = dict.fromkeys(labels_names)
    print labels
    for idx, i in enumerate(labels):
        labels[i] = idx
    labels.pop(u'удалено', None)
    return labels

labels = get_labels(raw['info'].labels_names)


epochs = mne.Epochs(raw_mne,events,event_id=labels,add_eeg_ref=False, preload = True, tmin = 0, tmax = 4.499)

{u'\u0443\u0434\u0430\u043b\u0435\u043d\u043e': None, u'\u043b\u0435\u0432\u0430\u044f': None, u'\u043e\u0442\u0434\u044b\u0445': None, u'\u043f\u0440\u0430\u0432\u0430\u044f': None}


In [16]:
epochs

<Epochs  |  n_events : 45 (all good), tmin : 0.0 (s), tmax : 4.499 (s), baseline : (None, 0), ~46.4 MB, data loaded,
 u'\u043b\u0435\u0432\u0430\u044f': 15, u'\u043e\u0442\u0434\u044b\u0445': 15, u'\u043f\u0440\u0430\u0432\u0430\u044f': 15>